#### Libraries

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pymavlog import MavLog
from datetime import datetime

#### Class initialization

In [23]:
class PixhawkLogger(MavLog):
    
    def __init__(self, log_path: str):
        super().__init__(log_path)
        self.message = 'None'
        self.parse()

    @staticmethod
    def timestamp_ms(TimeUS):
        timestamp = TimeUS / 1_000
        return pd.to_datetime(timestamp, unit='ms')

    @staticmethod
    def timestamp_s(TimeUS):
        timestamp = TimeUS / 1_000_000
        return pd.to_datetime(timestamp, unit='s')

    def get_message(self, message: str):
        self.message = message

    def get_field(self, field: str):
        return self.get(self.message)[field]

    def get_all_messages(self, debug: bool=False):
        all_messages = self._types
 
        if debug:
            print("-- Available Log Messages --")
            for message in all_messages:
                print(f"- {message}")

        return all_messages

    def get_all_fields(self, debug: bool = False):
        if self.message not in self._parsed_data:
            raise ValueError(f"[ERROR] No Message: {self.message}")
        
        fields = list(self._parsed_data[self.message].columns)

        if debug:
            print(f"-- Available Fields in [{self.message}] --")
            print(fields)
        
        return fields

    def get_statustext(self, debug: bool=False):
        self.message = 'MSG'
        statustext = self.get_field('Message') 

        if debug:
            for text in statustext:
                print(text)

        return statustext

    def get_gps_data(self):
        self.message = 'GPS'
        time = self.get_field('TimeUS')

        df_gps = pd.DataFrame({
            'time': self.timestamp_ms(time),
            'lat' : self.get_field('Lat'),
            'lon' : self.get_field('Lng'),
            'alt' : self.get_field('Alt')
        })

        return df_gps

    def get_att_data(self):
        self.message = 'ATT'
        time = self.get_field('TimeUS')

        df_att = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'roll'  : self.get_field('Roll'),
            'pitch' : self.get_field('Pitch'),
            'yaw'   : self.get_field('Yaw')
        })

        return df_att

    def get_ctun_data(self):
        self.message = 'CTUN'
        time = self.get_field('TimeUS')

        df_ctun = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'crt'  : self.get_field('CRt')
        })

        return df_ctun
    
    def get_imu_data(self):
        self.message = 'IMU'
        time = self.get_field('TimeUS')

        df_imu = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'gyrX'  : self.get_field('GyrX'),
            'gyrY'  : self.get_field('GyrY'),
            'gyrZ'  : self.get_field('GyrZ')
        })

        return df_imu



#### Object initialization

In [24]:
path = "log_1.bin"

logs = PixhawkLogger(path)

#### All available messages 

In [8]:
msg = logs.get_all_messages()

print(msg)

['UNIT', 'MULT', 'PARM', 'GPS', 'GPA', 'UBX1', 'UBX2', 'GRAW', 'GRXH', 'GRXS', 'SBPH', 'SBRH', 'SBRM', 'SBRE', 'MSG', 'RCIN', 'RCI2', 'RCOU', 'RCO2', 'RCO3', 'RSSI', 'BARO', 'BARD', 'PL', 'POWR', 'MCU', 'CMD', 'MAVC', 'RAD', 'CAM', 'TRIG', 'MNT', 'ARSP', 'BAT', 'BCL', 'MAG', 'MODE', 'RFND', 'DMS', 'BCN', 'PRX', 'PRXR', 'PM', 'SRTL', 'OABR', 'OADJ', 'SA', 'OAVG', 'SIM', 'TERR', 'ESC', 'CSRV', 'PIDR', 'PIDP', 'PIDY', 'PIDA', 'PIDS', 'PIDN', 'PIDE', 'DSTL', 'ACC', 'GYR', 'IMU', 'VIBE', 'ISBH', 'ISBD', 'RFRH', 'RFRF', 'RFRN', 'REV2', 'RSO2', 'RWA2', 'REV3', 'RSO3', 'RWA3', 'REY3', 'RISH', 'RISI', 'RASH', 'RASI', 'RBRH', 'RBRI', 'RRNH', 'RRNI', 'RGPH', 'RGPI', 'RGPJ', 'RMGH', 'RMGI', 'RBCH', 'RBCI', 'RVOH', 'ROFH', 'REPH', 'RSLL', 'REVH', 'RWOH', 'RBOH', 'XKF0', 'XKF1', 'XKF2', 'XKF3', 'XKF4', 'XKF5', 'XKFD', 'XKFM', 'XKFS', 'XKQ', 'XKT', 'XKTV', 'XKV1', 'XKV2', 'XKY0', 'XKY1', 'NKY0', 'NKY1', 'AHR2', 'AOA', 'ATT', 'ORGN', 'POS', 'RATE', 'ATSC', 'VSTB', 'MON', 'WDOG', 'RPM', 'FNCE', 'DSF', 

#### Message GPS

In [19]:
msg = 'GPS'

logs.get_message(msg)
logs.get_all_fields()

['timestamp',
 'TimeUS',
 'I',
 'Status',
 'GMS',
 'GWk',
 'NSats',
 'HDop',
 'Lat',
 'Lng',
 'Alt',
 'Spd',
 'GCrs',
 'VZ',
 'Yaw',
 'U']

#### Message CTUN

In [20]:
msg = 'CTUN'

logs.get_message(msg)
logs.get_all_fields()

['timestamp',
 'TimeUS',
 'ThI',
 'ABst',
 'ThO',
 'ThH',
 'DAlt',
 'Alt',
 'BAlt',
 'DSAlt',
 'SAlt',
 'TAlt',
 'DCRt',
 'CRt']

#### Message ATT

In [21]:
msg = 'ATT'

logs.get_message(msg)
logs.get_all_fields()

['timestamp',
 'TimeUS',
 'DesRoll',
 'Roll',
 'DesPitch',
 'Pitch',
 'DesYaw',
 'Yaw',
 'ErrRP',
 'ErrYaw',
 'AEKF']

#### Message IMU

In [22]:
msg = 'IMU'

logs.get_message(msg)
logs.get_all_fields()

['timestamp',
 'TimeUS',
 'I',
 'GyrX',
 'GyrY',
 'GyrZ',
 'AccX',
 'AccY',
 'AccZ',
 'EG',
 'EA',
 'T',
 'GH',
 'AH',
 'GHz',
 'AHz']

#### Dataframe GPS

In [25]:
logs.get_gps_data()

,time,lat,lon,alt
0,1970-01-01 00:07:48.004120,-7.265861,112.791125,17.20
1,1970-01-01 00:07:48.407139,-7.265861,112.791125,17.19
2,1970-01-01 00:07:48.607146,-7.265861,112.791125,17.22
3,1970-01-01 00:07:48.847163,-7.265861,112.791125,17.26
4,1970-01-01 00:07:48.987403,-7.265861,112.791125,17.28
...,...,...,...,...
455,1970-01-01 00:09:20.208642,-7.265860,112.791131,15.11
456,1970-01-01 00:09:20.408576,-7.265860,112.791131,15.11
457,1970-01-01 00:09:20.608641,-7.265860,112.791131,15.12
458,1970-01-01 00:09:20.848250,-7.265860,112.791131,15.09


#### Dataframe CTUN

In [27]:
logs.get_ctun_data()

,time,crt
0,1970-01-01 00:07:47.765516,0
1,1970-01-01 00:07:47.865576,0
2,1970-01-01 00:07:47.965894,0
3,1970-01-01 00:07:48.168956,0
4,1970-01-01 00:07:48.369558,0
...,...,...
923,1970-01-01 00:09:20.670603,3
924,1970-01-01 00:09:20.770576,4
925,1970-01-01 00:09:20.870573,4
926,1970-01-01 00:09:20.970584,4


#### Dataframe ATT

In [28]:
logs.get_att_data()

,time,roll,pitch,yaw
0,1970-01-01 00:07:47.692740,0.23,-0.94,109.45
1,1970-01-01 00:07:47.695727,0.23,-0.94,109.45
2,1970-01-01 00:07:47.697780,0.23,-0.94,109.45
3,1970-01-01 00:07:47.712830,0.23,-0.94,109.45
4,1970-01-01 00:07:47.715501,0.23,-0.94,109.45
...,...,...,...,...
36728,1970-01-01 00:09:21.157977,0.12,0.95,104.30
36729,1970-01-01 00:09:21.160212,0.12,0.95,104.30
36730,1970-01-01 00:09:21.162649,0.12,0.95,104.30
36731,1970-01-01 00:09:21.165151,0.12,0.95,104.30


#### Dataframe IMU

In [29]:
logs.get_imu_data()

,time,gyrX,gyrY,gyrZ
0,1970-01-01 00:07:47.692954,-0.001357,-0.003216,-0.000837
1,1970-01-01 00:07:47.695963,-0.001151,-0.003332,-0.000839
2,1970-01-01 00:07:47.698108,-0.000163,-0.003640,-0.000710
3,1970-01-01 00:07:47.713012,0.000485,-0.002796,-0.000200
4,1970-01-01 00:07:47.715640,-0.000112,-0.002723,-0.000036
...,...,...,...,...
36575,1970-01-01 00:09:21.158215,-0.000842,-0.003535,-0.001253
36576,1970-01-01 00:09:21.160351,-0.000695,-0.003007,-0.001205
36577,1970-01-01 00:09:21.162788,-0.000667,-0.002820,-0.001495
36578,1970-01-01 00:09:21.165290,-0.000684,-0.003115,-0.001595
